In [ ]:
# !pip install xlrd

In [ ]:
# pip install --no-index --find-links="xlrd-2.0.1-py2.py3-none-any.whl" xlrd

In [ ]:
# 'Life Ladder'                         生活階梯  
# 'Log GDP per capita'                  人均GDP的對數
# 'Social support'                      社會支持   
# 'Healthy life expectancy at birth'    出生時的健康預期壽命
# 'Freedom to make life choices'        生活選擇自由度
# 'Generosity'                          慷慨程度
# 'Perceptions of corruption'           腐敗感知程度 
# 'Positive affect'                     正面影響(幸福、歡笑和享受的平均水平)
# 'Negative affect'                     負面影響(擔心、悲傷和憤怒的平均水平)
# 'Confidence in national government'   對國家政府的信心
# 'Democratic Quality'                  民主品質(政府的民主程度)
# 'Delivery Quality'                    行政品質(政府的執行力)

In [2]:
# 第 1 部分
# 載入函式庫
import matplotlib.pyplot as plt
import numpy as np
import openensembles as oe 
import pandas as pd
from sklearn import metrics

# 載入資料集
data = pd.read_excel('../Data/WHR.xls')
regs = pd.read_excel('../Data/REG.xls')


# 使用 2017 年的資料並填補缺少的項目
recents = data[data.Year == 2017]
recents = recents.dropna(axis=1, how="all") # axis=1: 刪除包含缺失值的列; how="all": 所有的值都缺失，才刪除行或列
recents = recents.fillna(recents.median())

# 使用以下特徵
columns = ['Log GDP per capita',
           'Social support',
           'Healthy life expectancy at birth',
           'Freedom to make life choices',
           'Generosity',
           'Perceptions of corruption',
           'Positive affect',
           'Negative affect',
           'Confidence in national government',
           'Democratic Quality',
           'Delivery Quality']

# 建立 data 物件
cluster_data = oe.data(recents[columns], columns)

ModuleNotFoundError: No module named 'openensembles'

In [ ]:
recents.median()

In [ ]:
# 第 2 部分程式
# 建立集成模型
np.random.seed(123456)
results = {'K':[], 'size':[], 'silhouette': []}
# 測試不同的子群數
Ks = [2, 4, 6, 8, 10, 12, 14]
# 測試不同的基學習器個數
sizes = [5, 10, 20, 50]
for K in Ks:
    for ensemble_size in sizes:
        # 初始化基學習器
        ensemble = oe.cluster(cluster_data)
        for i in range(ensemble_size):
            # 訓練基學習器
            name = f'kmeans_{ensemble_size}_{i}'
            ensemble.cluster('parent', 'kmeans', name, K)

        # 使用共現鏈組成所有基學習器的輸出
        preds = ensemble.finish_co_occ_linkage(threshold = 0.5)
        print(f'K: {K}, size {ensemble_size}:', end=' ')
        # 計算輪廓係數
        silhouette = metrics.silhouette_score(recents[columns],
                                              preds.labels['co_occ_linkage'])
        print('%.2f' % silhouette)
        results['K'].append(K)
        results['size'].append(ensemble_size)
        results['silhouette'].append(silhouette)
        
results_df = pd.DataFrame(results)
cross = pd.crosstab(results_df.K,
                    results_df['size'], 
                    results_df['silhouette'],
                    aggfunc=lambda x: x)
print(cross)